In [ ]:
import descarteslabs as dl
import h5py
import requests
from shapely import geometry
from geojson import Feature, FeatureCollection
import json
import matplotlib.pyplot as plt
import geojson
import pickle
import os
import requests
from area import area
import datetime
from pyproj import Proj
import random
import glob


import networkx as nx

from scipy.optimize import curve_fit
from scipy.misc import factorial

from functools import partial
from shapely.ops import transform
from shapely.strtree import STRtree


from shapely.ops import transform,linemerge, unary_union, polygonize
from shapely.affinity import affine_transform
from functools import partial
import pyproj

from PIL import Image
import io
import numpy as np
from shp_utils_py3 import *

### Generate Images for Labelling

In [ ]:
catalog_client = dl.Catalog()
raster_client = dl.Raster()
metadata_client = dl.Metadata()
tasks = dl.Tasks()
vector_client = dl.Vector()
storage_client = dl.client.services.storage.Storage()

In [ ]:
API_KEY = '<your-api-key>'

In [ ]:
dir_name = 'FR'

In [ ]:
os.mkdir('./hand_verify_B/'+dir_name+'/')

In [ ]:
shp_str='US-'

In [ ]:
fnames = glob.glob('./results_fcs_additional/'+'FR'+'*.geojson')

In [ ]:
fnames = [f for f in fnames if not ('AU' in f or 'BR' in f) ]

In [ ]:
fnames

In [ ]:
run_fts = []
for f in fnames:
    run_fts += json.load(open(f,'r'))['features']

In [ ]:
run_fts = [ft for ft in run_fts if ft['properties']['confidence']=='B']

In [ ]:
len(run_fts)

In [ ]:
def img_poly(ft_poly,mmpix):

    #V_inv -> lat,lon
    #for each point, go centroid->pt
    #print(centroid)
    img_coords = []
    for pt in list(ft_poly.exterior.coords):
        #pt->lon,lat
        #centroid->lon,lat
        dist, angle, dummy = V_inv((ft_poly.centroid.y,ft_poly.centroid.x),(pt[1],pt[0]))
        dist=dist*1000
        #print ((400+2*(dist/mmpix*np.cos(2.*np.pi*(angle)/360.)),400+2*(dist/mmpix*np.sin(2*np.pi*angle/360.))))
        img_coords.append((400+2*(dist/mmpix*np.cos(2.*np.pi*(angle-90.)/360.)),400+2*(dist/mmpix*np.sin(2*np.pi*(angle-90.)/360.))))
    #V_inv(point1, point2
    #v_dir
    return img_coords

In [ ]:
def lonlat2pixXY(pt,dt):
    lon = pt[0]
    lat = pt[1]
    Y = (lat-dt[3]-dt[4]/dt[1]*(lon-dt[0]))/(dt[5]-dt[2]*dt[4]/dt[1])
    #print Y
    X = (lon-dt[0]-Y*dt[2])/dt[1]
    #print (lon,dt[0],X)
    #print X
    return [int(X),int(Y)]

In [ ]:
def inspect_sample(ft,src = 'google'):
    
    print (ft['properties'])
    ft_shp = geometry.shape(ft['geometry'])
    #print ('area',))
    print(ft_shp.type)
    if ft_shp.type=='MultiPolygon':
        run_polys = list(ft_shp)
    else:
        run_polys = [ft_shp]
        
    for ii_p,ft_poly in enumerate(run_polys):
        centroid = ft_poly.centroid ##lon,lat
        print ('centroid',centroid)
        ft_bbox = ft_poly.bounds
        
        Dx = ft_poly.bounds[2]-ft_poly.bounds[0]
        Dy = ft_poly.bounds[3]-ft_poly.bounds[1]
        print (max(abs(Dx),abs(Dy)))
        ft_box = geometry.box(*ft_poly.buffer(max(abs(Dx),abs(Dy))).bounds)
        
        
        fig, axs=plt.subplots(1,2,figsize=(24,12))

        if src=='google':
            box_sides = (V_inv((ft_bbox[1],ft_bbox[0]),(ft_bbox[1],ft_bbox[2]))[0]*1000,
                             V_inv((ft_bbox[1],ft_bbox[0]),(ft_bbox[3],ft_bbox[0]))[0]*1000)
            #print (box_sides)
            side_len = np.ceil(max(box_sides))
            print ('side_len (m)',side_len)

            zoom_dict = dict(zip(range(1,21),[156543.03392 * np.cos(centroid.xy[1][0] * np.pi / 180) / np.power(2, z) for z in range(1,21)]))
            #print (zoom_dict) ##<-- METERS PER PIX, not side length

            zoom = np.max(np.argwhere(np.array([(zoom_dict[k]*400-max(box_sides)) for k in range(1,21)])>0.))+1


            #min(zoom_dict.keys(), key=(lambda k: (zoom_dict[k]-max(box_sides))))
            print ('zoom',zoom, zoom_dict[zoom], 'area',area(ft['geometry']), np.array(centroid))
            pix_poly = img_poly(ft_poly,zoom_dict[zoom])

            urlstr = ''.join(["""https://maps.googleapis.com/maps/api/staticmap?center=""",
                        str(centroid.xy[1][0])+""","""+str(centroid.xy[0][0]),
                        """&zoom="""+str(zoom),
                        """&size=400x400&scale=2&maptype=satellite&format=png&key=""", str(API_KEY)])

            r = requests.get(urlstr, allow_redirects=True)
            #print (r.content)

            image_data = r.content
            image = Image.open(io.BytesIO(image_data))
            image = image.convert('RGB')

            arr = np.asarray(image)
            
            axs[0].imshow(arr)
            xs, ys = geometry.Polygon(pix_poly).exterior.xy
            #print (geometry.Polygon(pix_poly))
            #print (xs,ys)
            axs[0].plot(xs,ys, color='c', linewidth=2.)
            axs[0].set_title('Google Basemap (indeterminate date), {:.2f}m/px'.format(zoom_dict[zoom]), fontsize=20)
        
            
        elif src=='SPOT':
            scenes, ctx = dl.scenes.search(
            ft_box,
            products=['airbus:oneatlas:spot:v2'],
            start_datetime='2015-09-01',  end_datetime='2018-12-31', cloud_fraction=0.2, limit=10,
            )
            scenes = sorted(scenes, key=lambda k: k.properties.date, reverse=True)
            print ([s.properties.date for s in scenes])
            
            arr_SPOT = scenes[0].ndarray("red green blue", ctx)

            for s in scenes[1:]:
                fill_por = np.sum(arr_SPOT>0)/np.prod([*arr_SPOT.shape])
                #print (fill_por, np.sum(arr_S2), np.prod([*arr_S2.shape]))
                new_arr = s.ndarray("red green blue", ctx)
                #print ('shapes',new_arr.shape, np.sum(new_arr>0))

                arr_SPOT[new_arr.mask==False]=new_arr.data[new_arr.mask==False]


            scene_crs = scenes[0].properties['crs']
            dt = scenes[0].properties['geotrans']
            dt_shapely = [dt[1],dt[2],dt[4],dt[5],dt[0],dt[3]]

            WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

            wgs_proj = pyproj.Proj(WGS84)
            utm_proj = pyproj.Proj("+init="+scene_crs, preserve_units=True)


            dt[0] = utm_proj(*ctx.bounds[0:2])[0]
            dt[3] = utm_proj(*ctx.bounds[2:])[1]

            projection_func = partial(pyproj.transform, wgs_proj, utm_proj)

            utm_poly = transform(projection_func, ft_poly)
            pix_poly = geometry.Polygon([lonlat2pixXY(c,dt) for c in list(utm_poly.exterior.coords)])

            axs[0].imshow((np.swapaxes(np.swapaxes(arr_SPOT.data,0,2),0,1)/256).clip(0.,1.))
            xs,ys = pix_poly.exterior.xy
            axs[0].plot(xs,ys,c='c',linewidth=2)
            
            axs[0].set_title('SPOT {}, 10m/px, , {:d}x{:d}px'.format(scenes[0].properties.date,arr_SPOT.shape[1],arr_SPOT.shape[2]), fontsize=20)
        
            
        
        
        
        
        scenes, ctx = dl.scenes.search(
            ft_box,
            products=['sentinel-2:L1C'],
            start_datetime='2018-09-01',  end_datetime='2018-12-31', cloud_fraction=0.2, limit=10,
            )
        
        if len(scenes)>0:
            scenes = sorted(scenes, key=lambda k: k.properties.cloud_fraction, reverse=False)

            arr_S2 = scenes[0].ndarray("red green blue", ctx)

            for s in scenes[1:]:
                fill_por = np.sum(arr_S2>0)/np.prod([*arr_S2.shape])
                #print (fill_por, np.sum(arr_S2), np.prod([*arr_S2.shape]))
                new_arr = s.ndarray("red green blue", ctx)
                #print ('shapes',new_arr.shape, np.sum(new_arr>0))

                arr_S2[new_arr.mask==False]=new_arr.data[new_arr.mask==False]


            scene_crs = scenes[0].properties['crs']
            dt = scenes[0].properties['geotrans']
            dt_shapely = [dt[1],dt[2],dt[4],dt[5],dt[0],dt[3]]

            WGS84 = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

            wgs_proj = pyproj.Proj(WGS84)
            utm_proj = pyproj.Proj("+init="+scene_crs, preserve_units=True)


            dt[0] = utm_proj(*ctx.bounds[0:2])[0]
            dt[3] = utm_proj(*ctx.bounds[2:])[1]

            projection_func = partial(pyproj.transform, wgs_proj, utm_proj)

            utm_poly = transform(projection_func, ft_poly)
            pix_poly = geometry.Polygon([lonlat2pixXY(c,dt) for c in list(utm_poly.exterior.coords)])

            axs[1].imshow((np.swapaxes(np.swapaxes(arr_S2.data,0,2),0,1)/10000*2.5).clip(0.,1.))
            xs,ys = pix_poly.exterior.xy
            axs[1].plot(xs,ys,c='c',linewidth=2)
        axs[1].set_title('Sentinel-2 (Q4 2018), 1.5m/px, , {:d}x{:d}px'.format(arr_S2.shape[1],arr_S2.shape[2]), fontsize=20)
        fig.suptitle(ft['properties']['primary_id'], fontsize=26)
        #plt.tight_layout()
        fig.savefig('./hand_verify_B/'+dir_name+'/'+ft['properties']['primary_id']+'__'+str(ii_p)+'.png')
        plt.show()
        plt.close()
        
    
    return True


In [ ]:
for ft in run_fts[3:5]:
    inspect_sample(ft,src='SPOT')

In [ ]:
import multiprocessing as mp

print ('cpus: ',mp.cpu_count())

In [ ]:
deploy_fts = [(ft,) for ft in run_fts]

In [ ]:
len(deploy_fts)

In [ ]:
pool = mp.Pool(7)

jobs = []
#for i in range(5):
    

#for ii_ft,ft in enumerate(run_fts[0:20]):

#    print ('doing ii_ft',ii_ft)
#    p = mp.Process(target=inspect_sample, args=[ft,])
#    p.start()
#    p.join()
    
for res in pool.starmap(inspect_sample,deploy_fts[655:]):
    print (res)
    #get_trn_data(k,ii_k, TP_polys, FP_polys)
#for res in pool.starmap(fetch_tile_data,list(zip(dltiles['features'], range(len(dltiles['features']))))):
#    print (res)
    
#pickle.dump(res, open('res_test.pickle','wb'))

### Filter final images for v1.1

In [ ]:
### dir
out_str = 'CL'
labels_A = json.load(open('./S2_pics_A/'+out_str+'/labels.json','r')) 
labels_C = json.load(open('./S2_pics_C/'+out_str+'/labels.json','r')) 

In [ ]:
dn_labels_A = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_A if ll['label']==0]
dn_labels_C = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_C if ll['label']==0]

In [ ]:
print (len(dn_labels_A), len(labels_A), len(dn_labels_C), len(labels_C))

In [ ]:
fnames = sorted(glob.glob('./results_fcs_v1.0/CL*.geojson'))

In [ ]:
fnames

In [ ]:
all_fts = []
for f in fnames:
    all_fts+=json.load(open(f,'r'))['features']
    print (f, len(all_fts))

In [ ]:
dn_ii=[]
for ii_ft,ft in enumerate(all_fts):
    if ft['properties']['confidence']=='A':
        if ft['properties']['S2_ids_0'] in dn_labels_A:
            dn_ii.append(ii_ft)
    
    elif ft['properties']['confidence']=='C':
        if ft['properties']['S2_ids_0'] in dn_labels_C:
            dn_ii.append(ii_ft)

In [ ]:
len(dn_ii)

In [ ]:
out_fts = [ft for ii_ft, ft in enumerate(all_fts) if ii_ft not in dn_ii]

In [ ]:
len(all_fts), len(out_fts)

In [ ]:
json.dump(geojson.FeatureCollection(out_fts),open('./results_fcs_v1.1/'+out_str+'_v1.1.geojson','w'))

### Filter Images for v1.2

#### Get hand-made labels

In [ ]:
out_str = 'GB'
labels_B = json.load(open('./S2_pics_B/'+out_str+'/labels.json','r'))
labels_C = json.load(open('./S2_pics_C_gt1/'+out_str+'/labels.json','r'))

In [ ]:
keep_labels_B = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_B if ll['label']==1]
dn_labels_C = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_C if ll['label']==0]

In [ ]:
print (len(keep_labels_B), len(labels_B),len(dn_labels_C),len(labels_C))

#### Filter out 'B' confidence

In [ ]:
fnames = sorted(glob.glob('./results_fcs_v1.1/GB*.geojson'))

In [ ]:
fnames

In [ ]:
all_fts = []
for f in fnames:
    all_fts+=json.load(open(f,'r'))['features']
    print (f, len(all_fts))

In [ ]:
dn_ii=[]
for ii_ft,ft in enumerate(all_fts):
    if ft['properties']['confidence']=='B':
        if not ft['properties']['SPOT_ids_0'] in keep_labels_B:
            dn_ii.append(ii_ft)

In [ ]:
out_fts = [ft for ii_ft, ft in enumerate(all_fts) if ii_ft not in dn_ii]
print (len(out_fts))

#### Filter additional Cs then add them

In [ ]:
add_fs = sorted(glob.glob('./results_fcs_additional/GB*'))

In [ ]:
add_fs

In [ ]:
add_fts = []
for f in add_fs:
    add_fts+=json.load(open(f,'r'))['features']
    print (f, len(add_fts))

In [ ]:
add_fts = [ft for ft in add_fts if ft['properties']['primary_id'] not in dn_labels_C]

In [ ]:
len(add_fts)

In [ ]:
for ft in add_fts:
    properties = {
        'confidence': 'C',
        'install_date': ft['properties']['install_date'],
        'install_date_ints': ft['properties']['install_date_int'],
        'S2_ids_0': ft['properties']['primary_id'],
        'SPOT_ids_0': '',
        'area': area(json.dumps(ft['geometry']).replace('(','[').replace(')',']'))}
    out_fts.append(geojson.Feature(geometry=ft['geometry'],properties=properties))

In [ ]:
len(out_fts)

#### Resmash confidence Bs and Cs

In [ ]:
def get_matches_treenx(A_polys,P_polys):
    ### return a list of matches. both polys need a property which is primary_id
    
    P_tree = STRtree(P_polys)
    G = nx.Graph()
    
    for pp_A in A_polys:
        t_result = P_tree.query(pp_A)
        t_result = [pp for pp in t_result if pp.intersects(pp_A)]
        #print (len(t_result))
        if len(t_result)>0:
            for r in t_result:
                G.add_edge(pp_A.primary_id,r.primary_id)
                
    return list(nx.connected_components(G))

In [ ]:
def reduce2mp(polys, verbose=False):
    ### generate a big multipolygon -> takes forever. Make small mps of 100 each then, combine.
    if len(polys)==1:
        return polys[0]
    big_mps = []
    big_mp = polys[0]
    mod_count=0
    for ii in range(1,len(polys)):
        if ii%100==0:
            if verbose:
                print ('mod count',ii)
            mod_count+=1
            big_mps.append(big_mp)
            big_mp=polys[ii]
        else:
            #print (mod_count,ii)
            big_mp=big_mp.union(polys[ii])
    big_mps.append(big_mp)
            
    if verbose:
        print ('n big mps',len(big_mps))
    
    ### now reduce list of big_mps
    big_mp=big_mps[0]
    for ii in range(1,len(big_mps)):
        if verbose:
            print ('big_mp: ',ii)
        big_mp = big_mp.union(big_mps[ii])
    return big_mp

In [ ]:
B_polys = []
for ft in out_fts:
    if ft['properties']['confidence']=='B':
        pp = geometry.shape(ft['geometry'])
        pp.primary_id = ft['properties']['SPOT_ids_0']
        pp.install_date = ft['properties']['install_date']
        pp.install_date_int = ''
        B_polys.append(pp)
C_polys = []
for ft in out_fts:
    if ft['properties']['confidence']=='C':
        pp = geometry.shape(ft['geometry'])
        pp.primary_id = ft['properties']['S2_ids_0']
        pp.install_date = ft['properties']['install_date']
        pp.install_date_int = ''
        C_polys.append(pp)

print (len(B_polys), len(C_polys))
matches = get_matches_treenx(C_polys,B_polys)

print ('n matches',len(matches))
#print (matches)

new_A_fts = []
done_S2 = []
done_SPOT = []


for match in matches:
    print (match)
    m_polys_S2 = [pp for pp in C_polys if pp.primary_id in match]
    m_polys_SPOT = [pp for pp in B_polys if pp.primary_id in match]

    m_mp_S2 = reduce2mp(m_polys_S2)
    m_mp_SPOT = reduce2mp(m_polys_SPOT)

    intersection =m_mp_S2.intersection(m_mp_SPOT)
    union=m_mp_S2.union(m_mp_SPOT)

    IOU = area(json.dumps(geometry.mapping(intersection)).replace('(','[').replace(')',']')) / area(json.dumps(geometry.mapping(union)).replace('(','[').replace(')',']'))

    print ('IOU',IOU)

    if IOU>0.3:
        install_date_ints = ','.join(list(set([str(pp.install_date_int) for pp in m_polys_S2])))
        install_dates = ','.join(list(set([pp.install_date for pp in m_polys_S2])))

        properties_upload = {
                'confidence':'A',
                'install_date':install_dates,
                'install_date_ints':install_date_ints,
            }
        S2_ids = [pp.primary_id for pp in m_polys_S2]
        SPOT_ids = [pp.primary_id for pp in m_polys_SPOT]


        for index in range(int(len(S2_ids)/10)+1):
            properties_upload['S2_ids_'+str(index)] = ','.join(S2_ids[index*10:min(((index+1)*10),len(S2_ids))])

        for index in range(int(len(SPOT_ids)/7)+1):
            properties_upload['SPOT_ids_'+str(index)] = ','.join(SPOT_ids[index*7:min(((index+1)*7),len(SPOT_ids))])          


        #print ('install date',install_date_ints,install_dates)
        for pp_SPOT in m_polys_SPOT:
            properties_upload['area']=area(json.dumps(geometry.mapping(pp_SPOT)).replace('(','[').replace(')',']'))

            new_A_fts.append(geojson.Feature(
                                geometry=pp_SPOT.simplify(1e-5), 
                                properties=properties_upload
                            ))


        done_S2 += S2_ids
        done_SPOT += SPOT_ids

#add residual Cs
out_fts_trim = [ft for ft in out_fts if (ft['properties']['confidence']=='C') and (ft['properties']['S2_ids_0'] not in S2_ids)]
print ('resid Cs',len(out_fts_trim))

#add residual Bs
out_fts_trim += [ft for ft in out_fts if (ft['properties']['confidence']=='B') and (ft['properties']['SPOT_ids_0'] not in SPOT_ids)]
print ('residd Bs',len(out_fts_trim))

#add new As
out_fts_trim+=new_A_fts
print ('new As',len(out_fts_trim))

#add initial As
out_fts_trim+=[ft for ft in out_fts if ft['properties']['confidence']=='A']

print ('initial As',len(out_fts_trim), len(out_fts))

#### fix date int

In [ ]:
for ft in out_fts_trim:
    if ft['properties']['install_date']=='<2016-06':
        ft['properties']['install_date_ints']=0

#### Output

In [ ]:
json.dump(geojson.FeatureCollection(out_fts_trim),open('./results_fcs_v1.3/GB_v1.3.geojson','w'))

### Make memory efficient for CN

In [ ]:
### dir
out_str = 'CN'
labels_A = json.load(open('./S2_pics_A/'+out_str+'/labels.json','r')) 
labels_C = json.load(open('./S2_pics_C/'+out_str+'/labels.json','r')) 

In [ ]:
dn_labels_A = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_A if ll['label']==0]
dn_labels_C = [ll['fname'].split('__')[0].split('/')[-1] for ll in labels_C if ll['label']==0]

In [ ]:
dn_labels = dn_labels_A+dn_labels_C

In [ ]:
print (len(dn_labels_A), len(labels_A), len(dn_labels_C), len(labels_C))

In [ ]:
fnames = sorted(glob.glob('./results_fcs_v1.0/CN*.geojson'))

In [ ]:
fnames

In [ ]:
all_fts = []
tracker = 0
for f in fnames:
    new_fts =json.load(open(f,'r'))['features']
    tracker = len(all_fts)
    all_fts +=[ft for ft in new_fts if ft['properties']['confidence']=='B' or ft['properties']['S2_ids_0'] not in dn_labels]
    print (f, len(new_fts), len(all_fts)-tracker, len(all_fts))

In [ ]:
json.dump(geojson.FeatureCollection(all_fts),open('./results_fcs_v1.1/'+out_str+'_v1.1.geojson','w'))

### Collect all A and C

In [ ]:
fnames = sorted(glob.glob('./results_fcs_v1.1/*v1.1.geojson'))

In [ ]:
all_fts = []
for f in fnames:
    all_fts += [ft for ft in json.load(open(f,'r'))['features'] if ft['properties']['confidence'] in ['A','C']]
    print (f, len(all_fts))

In [ ]:
json.dump(geojson.FeatureCollection(all_fts),open('./results_fcs_v1.1/world_c_AC.geojson','w'))

### do labels for 1.3

In [ ]:
labels_a =  json.load(open(os.path.join('hand_labelling_client','cnx65a','labels.json'),'r'))
labels_b = json.load(open(os.path.join('hand_labelling_client','cnx65b','labels.json'),'r'))

In [ ]:
json.dump(labels_a+labels_b, open('./hand_labelling_client/cnx65/labels.json','w'))

In [ ]:
consol_fcs = glob.glob('./results_fcs_consolidated/*.geojson')

In [ ]:
consol_fcs

In [ ]:
windirs = ['e-e', 'f-f', 'g-g', 'h-h', 'i-i', 'j-j', 'k-k', 'l-l', 'm-m', 'n-n', 'o-o', 'p-p', 'q-q', 'r-r', 's-s', 'v-v', 'x-x']

In [ ]:
all_new_fts = []
for fc in consol_fcs:
    fts_fc = json.load(open(fc,'r'))['features']
    subdir = fc.split('/')[-1].split('.')[0].lower()
    labels =  json.load(open(os.path.join('hand_labelling_client',subdir,'labels.json'),'r'))
    
    if subdir in ['c-c','d-d']:
        for label in labels:
            label['fname'] = label['fname'].replace('꞉',':')
    elif subdir in windirs:
        for label in labels:
            label['fname'] = label['fname'].replace('--','-^').replace('-',':').replace('^','-')
    
    

    T_labels = [ll['fname'].replace('/','\\').split('\\')[-1].split('__')[0] for ll in labels if ll['label']==1]
    #try:
    #    print (fts_fc[0]['properties']['primary_id'],T_labels[0])
    #except:
    #    print ('nuh uh')
    new_fts = [ft for ft in fts_fc if ft['properties']['primary_id'] in T_labels]
    all_new_fts += new_fts
    
    print (fc, subdir, len(labels), len(T_labels), len(new_fts))

In [ ]:
len(all_new_fts)

In [ ]:
json.dump(geojson.FeatureCollection(all_new_fts),open('./results_fcs_additional/new_C_fts.geojson','w'))

### Do labels for 1.3 AR et al, KZ

In [ ]:
labels_ar =  json.load(open(os.path.join('hand_labelling_client','ar-et-al','labels.json'),'r'))
labels_kz = json.load(open(os.path.join('hand_labelling_client','kz-add','labels.json'),'r'))

In [ ]:
T_labels = [ll['fname'].replace('/','\\').split('\\')[-1].split('__')[0] for ll in labels_ar if ll['label']==1]

In [ ]:
T_labels +=[ll['fname'].replace('/','\\').split('\\')[-1].split('__')[0] for ll in labels_kz if ll['label']==1]

In [ ]:
ar_fts= json.load(open('./results_fcs_additional/additional_AR_NZ_CD_ID_ET.geojson','r'))['features']
kz_fts = json.load(open('./results_fcs_additional/additional_KZ.geojson','r'))['features']

In [ ]:
new_fts = [ft for ft in ar_fts if ft['properties']['primary_id'] in T_labels]
new_fts += [ft for ft in kz_fts if ft['properties']['primary_id'] in T_labels]

In [ ]:
json.dump(geojson.FeatureCollection(new_fts), open('./results_fcs_additional/filtered_ar_kz.geojson','w'))